In [ ]:
a = '12123123123123'
print(a[0:10])

In [ ]:
import requests
from langchain_community.llms.ollama import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json

def list_local_ollama_models():
    url = "http://127.0.0.1:11434/api/tags"  
    models_list = []
    try:
        response = requests.get(url)
        if response.status_code == 200:
            models = response.json()
            print("可用的Ollama模型列表：")
            for items in models.items():
                for item in items[1]:
                    model_name = item.get("name")
                    models_list.append(model_name)
        else:
            print(f"请求失败，状态码：{response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"请求错误：{e}")
    return models_list
list_local_ollama_models()

In [ ]:
def augment_multiple_query(query, numbers):
    # 扩展query，扩大数据集涉及区域
    template = """
            为模型提供指令，告诉它扮演一个优秀的外贸研究助理。
            指令包括基于提供的查询生成最多{numbers}个相关的额外查询问题。
            提示模型生成的问题应该简短，不包含复合句，并涵盖主题的不同方面。
            要确保问题是完整的，并且与原始查询相关联。
            输出格式为每行一个问题，不要对问题编号，输出中文。 
            Question: {query}
            """
    prompt = ChatPromptTemplate.from_template(template)
    llm = Ollama(model="qwen2:7b")
    chain = prompt | llm | StrOutputParser()
    extended_query = chain.invoke({"query": query, "numbers": numbers})
    # extended_queries = extended_query.strip().split("\n")
    extended_queries = [{"question": line.strip()} for line in extended_query.split('\n') if line.strip()]

    return extended_queries

In [ ]:
extended_queries = augment_multiple_query('进出口贸易是什么', 2)

In [ ]:
print(extended_queries)

In [ ]:
print(len(extended_queries))

In [ ]:
def chat_with_llm(extended_queries):
    llm = Ollama(model="qwen2:7b")
    responses = []
    template = """
                你是一个优秀的外贸研究助理，你现在需要回答我所给的问题
                回答需要详细、简练且专业，不超过200字，语气保持客观，以中文回答。
                {query}"""
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | llm | StrOutputParser()

    for dict in extended_queries:
        query = dict[0]
        print(query)
        response = chain.invoke({"query": query})
        responses.append(response)
    return json.dumps(responses, ensure_ascii=False)


In [ ]:
chat_with_llm(extended_queries)

In [ ]:
def chat_with_llm(extended_queries):
    # list = []
    list = ""
    for dict in extended_queries:
      # list += dict
      # list.append(query)
      query = dict["question"]
      list += query
    return list

chat_with_llm([ "1. 进出口贸易包括哪些主要环节？", "2. 国际贸易的主要影响因素有哪些？"])

In [ ]:
import json
questions = ['1. 进出口贸易中，如何评估和选择可靠的合作伙伴？', '2. 为了提高进出口效率，应该采取哪些关键策略和措施？']
rag_result = """
### 如何评估和选择可靠的合作伙伴？

在进口或出口业务中，选择一个可靠且合适的合作伙伴至关重要。首先，应通过专业平台、行业展会及商业数据库获取潜在供应商信息，并进行初步筛选。其次，深入了解其经营历史、财务状况、产品质量认证（如ISO、CE等）以及客户反馈评价。实地考察公司运营情况也是关键一步。最后，签订合作前需详细商谈合同条款，包括交货时间、价格稳定性、售后服务等，确保权益得到充分保障。

### 如何提高进出口效率？

提高进口出口的效率通常涉及优化供应链管理流程、提升信息技术应用和加强与合作伙伴的合作关系。首先采用先进的物流管理系统和自动化技术（如RFID、区块链），以实现订单跟踪、库存管理和运输路线优化。其次，通过数字化平台进行实时数据交换，减少信息不对称带来的延误。同时，建立稳定的供应链网络，并定期评估和调整策略来应对市场变化。最后，强化与供应商的沟通协作，确保供需匹配和快速响应需求变动，从而提升整体运营效率。
"""
llm_result = """
### 评估和选择可靠合作伙伴

在进出口贸易中，评估和选择可靠合作伙伴是至关重要的。首先，应通过公开的商业数据库、行业协会或第三方信用评价系统核查其基本信息及过往业绩；其次，了解其财务状况稳定性、运营历史以及市场声誉；同时，关注对方的生产、物流和服务能力是否满足需求。此外，有效的沟通机制和共同的业务理念也是重要考量因素。最终选择与具有良好信誉、稳定供货能力和合适价格策略的合作方建立长期合作关系。

### 提高进出口效率的关键策略和措施

提高进出口贸易效率，可采取以下策略：
1. **数字化转型**：利用电子数据交换（EDI）、云计算、物联网等技术优化流程，减少纸质文件处理时间。
2. **供应链整合**：与供应商、物流商等建立紧密合作关系，实现信息共享和同步运作，降低协调成本。
3. **风险管理**：通过多元化供应来源、采用保险和信用证等方式分散风险，确保贸易连续性。
4. **优化库存管理**：采用精益生产或JIT（Just in Time）策略，减少库存成本和过期风险。
5. **强化法律法规合规**：深入了解并遵循各国的进出口法规与政策要求，避免法律纠纷。
6. **利用专业服务**：通过聘请国际贸易顾问、律师等专业人士提供咨询和服务，提升决策质量。

这些措施能有效提高贸易效率，降低运营风险，并增强市场竞争力。
"""
def post_processing(questions, rag_result, llm_result):
    rag_answers = [answer.strip().replace('\n', '') for answer in rag_result.split('\n')]
    llm_answers = [answer.strip().replace('\n', '') for answer in llm_result.split('\n')]
    print(len(questions),len(rag_answers),len(llm_answers))
    # if len(questions) != len(rag_answers) or len(questions) != len(llm_answers):
    #     raise ValueError("The number of questions does not match the number of answers.")

    data = []
    for i in range(len(questions)):
        entry = {
            "question": [
                {
                    "from": "human",
                    "value": questions[i].strip()
                }
            ],
            "chosen": {
                "from": "gpt",
                "value": rag_answers[i]
            },
            "rejected": {
                "from": "gpt",
                "value": llm_answers[i]
            }
        }

        data.append(entry)
    # 将结果转换为 JSON 格式的字符串
    json_result = json.dumps(data, ensure_ascii=False)
    return json_result
post_processing(questions, rag_result, llm_result)

In [2]:
original_query = "推动北京财政收入增长的因素是什么?"
augmented_queries = ["123","23123"]
queries = [original_query] + augmented_queries
queries

['推动北京财政收入增长的因素是什么?', '123', '23123']

In [1]:
import jieba
from collections import Counter
import language_tool_python
import numpy as np

# 初始化语法检查工具
tool = language_tool_python.LanguageTool('zh-CN')

# 计算语法错误数量
def grammar_errors(text):
    matches = tool.check(text)
    return len(matches)

# 计算词汇丰富度（TTR）
def type_token_ratio(text):
    tokens = jieba.lcut(text)
    return len(set(tokens)) / len(tokens)

# 计算关键词匹配
def keyword_matching(question, answer):
    question_tokens = set(jieba.lcut(question))
    answer_tokens = set(jieba.lcut(answer))
    common_tokens = question_tokens & answer_tokens
    return len(common_tokens) / len(question_tokens)

# 计算句子长度分布
def sentence_length_distribution(text):
    sentences = text.split('。')
    lengths = [len(jieba.lcut(sentence)) for sentence in sentences if sentence]
    return lengths

# 计算词频分布
def word_frequency(text):
    tokens = jieba.lcut(text)
    return Counter(tokens)

# 示例问题和回答
question = "国际贸易中的经济、社会和环境趋势是什么？"
answer1 = "国际贸易中的经济趋势包括全球化的持续发展。"
answer2 = "国际贸易中的经济趋势包括全球化和数字贸易的发展。"

# 计算文本质量和相关性指标
grammar1 = grammar_errors(answer1)
grammar2 = grammar_errors(answer2)

ttr1 = type_token_ratio(answer1)
ttr2 = type_token_ratio(answer2)

keyword_match1 = keyword_matching(question, answer1)
keyword_match2 = keyword_matching(question, answer2)

lengths1 = sentence_length_distribution(answer1)
lengths2 = sentence_length_distribution(answer2)

freq_q = word_frequency(question)
freq_a1 = word_frequency(answer1)
freq_a2 = word_frequency(answer2)

def freq_similarity(freq1, freq2):
    # 计算两个词频分布的余弦相似度
    common_keys = set(freq1.keys()).union(set(freq2.keys()))
    vec1 = np.array([freq1.get(key, 0) for key in common_keys])
    vec2 = np.array([freq2.get(key, 0) for key in common_keys])
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

freq_sim1 = freq_similarity(freq_q, freq_a1)
freq_sim2 = freq_similarity(freq_q, freq_a2)

print("文本质量和相关性比较：")
print(f"回答1 - 语法错误数: {grammar1}, TTR: {ttr1:.4f}, 关键词匹配: {keyword_match1:.4f}, 句子长度: {lengths1}, 词频相似度: {freq_sim1:.4f}")
print(f"回答2 - 语法错误数: {grammar2}, TTR: {ttr2:.4f}, 关键词匹配: {keyword_match2:.4f}, 句子长度: {lengths2}, 词频相似度: {freq_sim2:.4f}")



Unzipping /tmp/tmpjbwd_8p0.zip to /home/mth/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /home/mth/.cache/language_tool_python.
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.694 seconds.
Loading model cost 0.694 seconds.
Prefix dict has been built successfully.
Prefix dict has been built successfully.


文本质量和相关性比较：
回答1 - 语法错误数: 0, TTR: 0.9000, 关键词匹配: 0.2500, 句子长度: [9], 词频相似度: 0.3333
回答2 - 语法错误数: 0, TTR: 0.9167, 关键词匹配: 0.3333, 句子长度: [11], 词频相似度: 0.3858


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# 加载GPT-2模型和tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def calculate_perplexity(text):
    # 对文本进行编码
    encodings = tokenizer(text, return_tensors='pt')
    input_ids = encodings.input_ids
    
    # 禁用梯度计算
    with torch.no_grad():
        # 计算模型输出
        outputs = model(input_ids, labels=input_ids)
        
        # 获取交叉熵损失
        loss = outputs.loss
        perplexity = torch.exp(loss).item()
    
    return perplexity

# 示例回答
answer1 = "There are several trends in international trade. Economically, globalization continues to integrate markets."
answer2 = "Economic trends in international trade include globalization and digital trade."

# 计算Perplexity
ppl1 = calculate_perplexity(answer1)
ppl2 = calculate_perplexity(answer2)

print(f"Perplexity of Answer 1: {ppl1:.2f}")
print(f"Perplexity of Answer 2: {ppl2:.2f}")


In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import math
import jieba
import pandas as pd
from collections import Counter
import language_tool_python
import numpy as np
from rouge_score import rouge_scorer

path = '/share148/huggingface/models--Qwen--Qwen2-7B-Instruct/snapshots/41c66b0be1c3081f13defc6bdf946c2ef240d6a6'
model = AutoModelForCausalLM.from_pretrained(path)
tokenizer = AutoTokenizer.from_pretrained(path)
tool = language_tool_python.LanguageTool('zh-CN')


class AnswerEvaluator:
    def __init__(self):
        self.grammar_scores1 = []
        self.grammar_scores2 = []
        
        self.ttr_scores1 = []
        self.ttr_scores2 = []
        
        self.keyword_matches1 = []
        self.keyword_matches2 = []
        
        self.lengths_distributions1 = []
        self.lengths_distributions2 = []
        
        self.freq_similarities1 = []
        self.freq_similarities2 = []
        
        self.perplexities1 = []
        self.perplexities2 = []

    def evaluate(self, question, answer1, answer2):
        for i in range(len(question)):
            self.grammar_scores1.append(self.grammar_errors(answer1[i]))
            self.grammar_scores2.append(self.grammar_errors(answer2[i]))

            self.ttr_scores1.append(self.type_token_ratio(answer1[i]))
            self.ttr_scores2.append(self.type_token_ratio(answer2[i]))

            self.keyword_matches1.append(self.keyword_matching(question[i], answer1[i]))
            self.keyword_matches2.append(self.keyword_matching(question[i], answer2[i]))

            self.lengths_distributions1.append(self.sentence_length_distribution(answer1[i]))
            self.lengths_distributions2.append(self.sentence_length_distribution(answer2[i]))

            freq_q = self.word_frequency(question[i])
            freq_a1 = self.word_frequency(answer1[i])
            freq_a2 = self.word_frequency(answer2[i])

            self.freq_similarities1.append(self.freq_similarity(freq_q, freq_a1))
            self.freq_similarities2.append(self.freq_similarity(freq_q, freq_a2))

            self.perplexities1.append(self.calculate_perplexity(answer1[i]))
            self.perplexities2.append(self.calculate_perplexity(answer2[i]))
    
    def round_results(self, results, decimal_places=2):
        def round_value(value):
            if isinstance(value, list):
                return [round(v, decimal_places) if isinstance(v, (int, float)) else v for v in value]
            return round(value, decimal_places)

        rounded_results = {}
        for key, value in results.items():
            rounded_results[key] = [round_value(sublist) for sublist in value]
        return rounded_results

    # 计算语法错误数量
    def grammar_errors(self, text):
        matches = tool.check(text)
        return len(matches)

    # 计算词汇丰富度（TTR）
    def type_token_ratio(self, text):
        tokens = jieba.lcut(text)
        return len(set(tokens)) / len(tokens)

    # 计算关键词匹配
    def keyword_matching(self, question, answer):
        question_tokens = set(jieba.lcut(question))
        answer_tokens = set(jieba.lcut(answer))
        common_tokens = question_tokens & answer_tokens
        return len(common_tokens) / len(question_tokens)

    # 计算句子长度分布
    def sentence_length_distribution(self, text):
        sentences = text.split('。')
        lengths = [len(jieba.lcut(sentence)) for sentence in sentences if sentence]
        return lengths

    # 计算词频分布
    def word_frequency(self, text):
        tokens = jieba.lcut(text)
        return Counter(tokens)

    def calculate_perplexity(self, text):
        # 对文本进行编码
        encodings = tokenizer(text, return_tensors='pt')
        input_ids = encodings.input_ids
        
        # 禁用梯度计算
        with torch.no_grad():
            # 计算模型输出
            outputs = model(input_ids, labels=input_ids)
            
            # 获取交叉熵损失
            loss = outputs.loss
            perplexity = math.exp(loss.item())
        
        return perplexity

    def freq_similarity(self, freq1, freq2):
        # 计算两个词频分布的余弦相似度
        common_keys = set(freq1.keys()).union(set(freq2.keys()))
        vec1 = np.array([freq1.get(key, 0) for key in common_keys])
        vec2 = np.array([freq2.get(key, 0) for key in common_keys])
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

    def calaculate(self):
        results = {
                # "grammar_scores": [self.grammar_scores1, self.grammar_scores2],
                # "ttr_scores": [self.ttr_scores1, self.ttr_scores2],
                "keyword_matches": [self.keyword_matches1, self.keyword_matches2],
                # "lengths_distributions": [self.lengths_distributions1, self.lengths_distributions2],
                "freq_similarities": [self.freq_similarities1, self.freq_similarities2],
                "perplexities": [self.perplexities1, self.perplexities2],
                # "distinct1_scores": [self.distinct1_scores1, self.distinct1_scores2],
                # "distinct2_scores": [self.distinct2_scores1, self.distinct2_scores2]
                }
        rounded_results = self.round_results(results)
        values = [value for value in rounded_results.values()]
        rag_scores = []
        llm_scores = []

        for pair in values:
            # 提取rag_score
            rag_score = pair[0]
            rag_scores.append(rag_score)

            # 提取llm_score
            llm_score = pair[1]
            llm_scores.append(llm_score)

        print(values)
        output  = pd.DataFrame({
        "Metric": rounded_results.keys(),
        "RAG_Score": rag_scores,
        "LLM_Socre": llm_scores
    })
        return output

# 示例文本
questions = ["什么是国际贸易？", "全球化的影响是什么？"]
answers1 = ["国际贸易是国家之间的商品和服务交换。", "全球化促进了经济增长。"]
answers2 = ["国际贸易涉及多个国家的经济活动。", "全球化带来了文化交流。"]

evaluator = AnswerEvaluator()

# 进行评估
evaluator.evaluate(questions, answers1, answers2)

# 获取结果
results = evaluator.calaculate()
# 计算ROUGE分数
print(results)

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.82it/s]


[[[0.5, 0.17], [0.25, 0.17]], [[0.32, 0.17], [0.18, 0.18]], [[7.38, 657.05], [22.03, 611.0]]]
              Metric       RAG_Score       LLM_Socre
0    keyword_matches     [0.5, 0.17]    [0.25, 0.17]
1  freq_similarities    [0.32, 0.17]    [0.18, 0.18]
2       perplexities  [7.38, 657.05]  [22.03, 611.0]


In [5]:
rag_result = """
商品命名应考虑国际市场文化差异，选用易理解、记忆的名称；突出产品特性或产地优势吸引消费者；避免使用可能引起误解或禁忌的词汇；可结合品牌故事或历史人物增强情感联系；适时采用外来词满足求新心理。<eos>

在外贸中，商品命名应遵循以下原则：1. 准确性，确保名称能清晰反映商品特性；2. 国际化，避免使用地方方言或难译词汇；3. 法律合规，不侵犯商标权，遵守各国法规；4. 易于记忆和拼写，便于全球客户识别与传播；5. 营销导向，体现产品卖点，吸引目标市场。<eos>
"""
rag_answers = [answer.strip().replace('\n', '') for answer in rag_result.split('<eos>')]
# for i in range(len(rag_answers)):
print(rag_answers[2])

In [6]:
my_dict = {'a': 1, 'b': 2, 'c': 3}
values = my_dict.values()
print(list(values))

[1, 2, 3]


In [20]:
values = [[[0.5, 0.17], [0.25, 0.17]], [[0.32, 0.17], [0.18, 0.18]], [[7.38, 657.05], [22.03, 611.0]]]
rag_scores = []
llm_scores = []

for pair in values:
    # 提取rag_score
    rag_score = pair[0]
    rag_scores.append(rag_score)

    # 提取llm_score
    llm_score = pair[1]
    llm_scores.append(llm_score)

print(rag_scores, llm_scores) 
# 给定的数据


[[0.5, 0.17], [0.32, 0.17], [7.38, 657.05]] [[0.25, 0.17], [0.18, 0.18], [22.03, 611.0]]


In [7]:
from utils import *
llm = Ollama(model="qwen2:7b")
embedding_function = get_embeddings_function("zh")
generated_prompts = []
template = """
            请扮演一位优秀的外贸研究助理。我将提供与外贸领域相关的文本，请你按照以下步骤操作：
            1.你必须参考我给你的检索内容{context}，这些内容已按照与文本主旨相关性进行了降序排列，越靠前的主题越相关。
            2.根据排序文本概括这段文本内容，并根据此生成{number}个相关问题。这些问题的答案必须存在于文本中，并且与原始文本紧密相关。
            3.确保每个问题都是完整的句子，且两个问题不相关；只输出问题本身，不需要提供答案、分析或总结。
            4.生成的问题应该简短明了，避免使用复合句。
            特别注意：不要对问题进行编号，输出中文问题即可。
            例如，一个合格的问题可能是：“命名商品的方法有哪些？” 请仅返回问题列表
            """
PROMPT = PromptTemplate.from_template(template=template)
path = "/data/share9/XPT/dataset/dataset_8/book/data/进出口贸易实务教程 第八版-540.jsonl"
with open(path, "r", encoding='utf-8') as f:
    data = []
    for line in f:
        data.append(json.loads(line))
for data in data:
    string = ""
    string += data["text"]

print(len(string))

searching_blob = Blob.from_path(path)

parser = MyParser()
output = parser.lazy_parse(searching_blob)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=200, add_start_index=True,
                                                length_function=len,is_separator_regex=False)
output = text_splitter.split_documents(output)



534842


In [31]:
print(output[0])
print(type(output[0]))
n = len(output)
print(n)

page_content='{"text": "商品的名称和质量冏 品的名称和质量是国际货物买卖当事人双方首先需要商定的交易条件，是买卖双方 进行交易的物质基础。如果商品的名称和质量不明确,买卖双方也就失去了洽商的依据， 无法开展交易。因此，商品的名称和质量是国际货物买卖合同中的主要条款之一。\n第一节商品的名称商品的名称(name of commodity)，或称“品名”，是指能使某种商品区别于其他商品的\t一空_一种称呼或概念。商品的名称可区分为通用名称和特定名称。通用名称通常是某一类商 品的总称,用于区分不同种类的商品;而特定名称则用于区别同种类别商品的特定类型。\n商品的名称在一定程度上体现了商品的自然属性、用途以及主要的性能特征。加工程度 低的商品，其名称一般较多地反映该商品所具有的自然属性,加工程度越高，商品的名称 也越多地体现出该商品的性能特征。命名商品的目的在于使商品的名称与消费者心理相 吻合，便于记忆、激发联想、规避禁忌、名实一体。命名商品的方法有许多，概括起来，主要 有以下几种。\n1. 以主要用途命名。这种方法突出商品的性能和用途,便于消费者认知并按其需要 购买，主要用于日用工业品、化妆品和医药品等。如织布机、旅游鞋、杀虫剂、护手霜等。\n2. 以所使用的主要原材料或成分命名。这种方法通过突出商品所使用的主要原材 料或名贵成分，便于消费者了解商品的质量。如棉布、涤纶纱、羊毛衫、铝锅、玻璃杯、陶瓷 杯、冰糖燕窝、西洋参蜂王浆、人参珍珠霜等。\n3. 以外观造型命名。以商品的外观造型命名商品的方法有利于消费者直观地从字 义上了解该商品的特征，主要用于命名食品、工艺品等商品。如绿豆、黄豆、喇叭裤、直筒 裤、宝塔纱、纸管等。\n4. 以商品的产地命名。以产地命名主要是由于特定商品历史悠久，声誉与其产地紧 密相关，冠以产地名称可以突出该商品的地方特色,增强其对消费者的吸引力。例如，贵=——__Z州茅台酒、金华火腿、云南白药、西湖龙井、青岛啤酒、镇江香醋等。\n5. 以人物名字命名。即以著名的历史人物或传说中的人物命名，其目的在于引起消 费者睹物思人，引起丰富的情感联系和文化联想，如孔府家酒、李宁等。\n6. 以企业名称命名。这种方法通过突出商品生产者的字号和信誉，加深消费者对企 业的认识,获得最佳的传播效果。如，索尼、日立、东芝、苹果、华为、三星等,都是

In [3]:
n = len(output)
results = ""
content = output
print((content))
vector_db = Chroma.from_documents(content, embedding=embedding_function)   #创建向量数据库
retriever = vector_db.as_retriever(search_kwargs={"k": 10})

# compressor = FlashrankRerank()
# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=compressor, base_retriever=retriever
# )
docs = retriever.invoke("请分析文本并提取其核心内容。文本中可能存在诸如“这是一段随机文本”之类的噪声，这些内容与主旨无关，请在概括时忽略这些噪声。请确保返回的概括简洁明了，突出文本的主要观点。？")
docs


[Document(metadata={'line_number': 1, 'source': '/data/share9/XPT/dataset/dataset_8/book/data/进出口贸易实务教程 第八版-540.jsonl', 'start_index': 0}, page_content='{"text": "商品的名称和质量冏 品的名称和质量是国际货物买卖当事人双方首先需要商定的交易条件，是买卖双方 进行交易的物质基础。如果商品的名称和质量不明确,买卖双方也就失去了洽商的依据， 无法开展交易。因此，商品的名称和质量是国际货物买卖合同中的主要条款之一。\\n第一节商品的名称商品的名称(name of commodity)，或称“品名”，是指能使某种商品区别于其他商品的\\t一空_一种称呼或概念。商品的名称可区分为通用名称和特定名称。通用名称通常是某一类商 品的总称,用于区分不同种类的商品;而特定名称则用于区别同种类别商品的特定类型。\\n商品的名称在一定程度上体现了商品的自然属性、用途以及主要的性能特征。加工程度 低的商品，其名称一般较多地反映该商品所具有的自然属性,加工程度越高，商品的名称 也越多地体现出该商品的性能特征。命名商品的目的在于使商品的名称与消费者心理相 吻合，便于记忆、激发联想、规避禁忌、名实一体。命名商品的方法有许多，概括起来，主要 有以下几种。\\n1. 以主要用途命名。这种方法突出商品的性能和用途,便于消费者认知并按其需要 购买，主要用于日用工业品、化妆品和医药品等。如织布机、旅游鞋、杀虫剂、护手霜等。\\n2. 以所使用的主要原材料或成分命名。这种方法通过突出商品所使用的主要原材 料或名贵成分，便于消费者了解商品的质量。如棉布、涤纶纱、羊毛衫、铝锅、玻璃杯、陶瓷 杯、冰糖燕窝、西洋参蜂王浆、人参珍珠霜等。\\n3. 以外观造型命名。以商品的外观造型命名商品的方法有利于消费者直观地从字 义上了解该商品的特征，主要用于命名食品、工艺品等商品。如绿豆、黄豆、喇叭裤、直筒 裤、宝塔纱、纸管等。\\n4. 以商品的产地命名。以产地命名主要是由于特定商品历史悠久，声誉与其产地紧 密相关，冠以产地名称可以突出该商品的地方特色,增强其对消费者的吸引力。例如，贵=——__Z州茅台酒、金华火腿、云南白药、西湖龙井、青岛啤酒、镇江香醋等。\\n5. 

[Document(metadata={'line_number': 1, 'source': '/data/share9/XPT/dataset/dataset_8/book/data/进出口贸易实务教程 第八版-540.jsonl', 'start_index': 291854}, page_content='品目录、价目单和其他宣传品的行为区分开来。在后一种情况下，广告的对象是广大社会 公众，商品目录、价目单和其他宣传品是普遍寄发给为数众多的客商的。这些对象都不属 特定的人。因此，这类行为一般不能构成发盘,而仅能视为发盘邀请。但是，如果商业广 告或宣传品内容十分具体、明确和肯定，符合发盘规定的，视为发盘,④一旦见到广告或宣参见《中华人民共和国合同法》第14条(二)款。\\n摘译自 John O.Honnold, Uniform Law for International Sales under 1980 United Nations Convention, Kluwer Law & Taxation Publishers, Deventer 1989 o参见《中华人民共和国合同法》第15条。\\n参见《中华人民共和国合同法》第15条(《中华人民共和国民法典》第473条)。\\n传品的人作出接受的行为,该刊登广告者即须按广告中所提出的条件，履行其诺言。①例 如,外国某商行在报上刊登广告贱卖裘皮大衣，每件100美元(原价1。00美元)，规定了 售货地点、日期和营业时间。见到该广告的社会公众按时前往购货,该商行必须按在广告 中所作出的诺言廉价出售裘皮大衣。至于向国外众多客商大量寄发商品目录、价目单等 宣传品，谨慎的出口商往往在这些宣传品上注明“所列价格仅供参考”(The prices stated are for reference only)、“价格需经确认为准”(The prices shall be subject to confirmation)或 “价格不经事先通知得予变动” (The prices may be altered without prior notice )等字句，以免 因市价上涨而国外收件人要求按宣传品上所列价格订约，使自己处于被动的境地,或由此 造成经济损失。\\n3. 内容十分确定。《公约》第14条规定:一项订立合同的建议“如果写明货

In [4]:
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)
chain = create_stuff_documents_chain(llm, PROMPT)
result = chain.invoke({"context": reordered_docs, "number": 10, "query": "请按要求生成提问"}) 
results += result + '\n'
print(results)
# generated_prompts.append(result)

1. 在国际贸易中如何确保书面合同的合法性和有效性？
2. 制定有效的发盘时需要考虑哪些主要交易条件？

